In [189]:
import numpy as np
import re
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [190]:
import contractions

In [270]:
filename = "dune-no-appendix-or-quote.txt"

with open(filename, ) as f:
    text = f.read()

In [271]:
token_type = 'word'

In [272]:
start = text.find("= = = = = =")
text = text[start:]
text = text.replace('\\', ' ')
text = text.replace('\n', ' ')
text = text.replace(r"\'","'")

In [173]:
seq_length = 20
start_story = '| '*seq_length

In [273]:
text = text[13:]

In [274]:
text = text.replace('  page','')

In [275]:
text = text.replace('. . .', '...')

In [276]:
text = text.split('= = = = = =')

In [310]:
text[33] = text[33][5:]

In [312]:
text[2] = text[2][5:]

In [ ]:
text

In [176]:

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False)
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
    
    
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]


In [177]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'of': 3,
 'to': 4,
 'and': 5,
 'he': 6,
 'in': 7,
 'his': 8,
 'said': 9,
 'you': 10,
 'it': 11,
 'that': 12,
 'i': 13,
 'she': 14,
 'was': 15,
 'at': 16,
 'this': 17,
 'on': 18,
 'paul': 19,
 'with': 20,
 'for': 21,
 'her': 22,
 'had': 23,
 'is': 24,
 'him': 25,
 'be': 26,
 'as': 27,
 'but': 28,
 'my': 29,
 'from': 30,
 'jessica': 31,
 'duke': 32,
 'not': 33,
 'we': 34,
 'they': 35,
 'there': 36,
 'have': 37,
 'thought': 38,
 'one': 39,
 'your': 40,
 'them': 41,
 'baron': 42,
 'what': 43,
 'out': 44,
 'hawat': 45,
 'by': 46,
 'kynes': 47,
 'me': 48,
 'an': 49,
 'will': 50,
 'into': 51,
 'no': 52,
 'could': 53,
 'were': 54,
 'are': 55,
 'all': 56,
 'their': 57,
 'fremen': 58,
 'do': 59,
 'man': 60,
 'here': 61,
 'now': 62,
 'know': 63,
 'been': 64,
 'asked': 65,
 'must': 66,
 'if': 67,
 'down': 68,
 'back': 69,
 'up': 70,
 'like': 71,
 'see': 72,
 'can': 73,
 'over': 74,
 'then': 75,
 'leto': 76,
 'our': 77,
 "it's": 78,
 'about': 79,
 'hand': 80,
 'how': 81,
 'eyes

In [178]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    

    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)


Number of sequences: 93680 



In [179]:
X.shape

(93680, 20)

In [180]:
y.shape

(93680, 8473)

In [181]:
print(tokenizer.word_index)
print(token_list)

{'the': 1, 'a': 2, 'of': 3, 'to': 4, 'and': 5, 'he': 6, 'in': 7, 'his': 8, 'said': 9, 'you': 10, 'it': 11, 'that': 12, 'i': 13, 'she': 14, 'was': 15, 'at': 16, 'this': 17, 'on': 18, 'paul': 19, 'with': 20, 'for': 21, 'her': 22, 'had': 23, 'is': 24, 'him': 25, 'be': 26, 'as': 27, 'but': 28, 'my': 29, 'from': 30, 'jessica': 31, 'duke': 32, 'not': 33, 'we': 34, 'they': 35, 'there': 36, 'have': 37, 'thought': 38, 'one': 39, 'your': 40, 'them': 41, 'baron': 42, 'what': 43, 'out': 44, 'hawat': 45, 'by': 46, 'kynes': 47, 'me': 48, 'an': 49, 'will': 50, 'into': 51, 'no': 52, 'could': 53, 'were': 54, 'are': 55, 'all': 56, 'their': 57, 'fremen': 58, 'do': 59, 'man': 60, 'here': 61, 'now': 62, 'know': 63, 'been': 64, 'asked': 65, 'must': 66, 'if': 67, 'down': 68, 'back': 69, 'up': 70, 'like': 71, 'see': 72, 'can': 73, 'over': 74, 'then': 75, 'leto': 76, 'our': 77, "it's": 78, 'about': 79, 'hand': 80, 'how': 81, 'eyes': 82, 'us': 83, 'water': 84, 'who': 85, 'so': 86, 'saw': 87, 'arrakis': 88, 'roo

In [183]:
load_saved_model = False

if load_saved_model:
    # model = load_model('./saved_models/lstm_aesop_1.h5')
    model = load_model('./saved_models/aesop_dropout_100.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    # x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = RMSprop(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [184]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 100)         847300    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_4 (Dense)              (None, 8473)              2177561   
Total params: 3,390,429
Trainable params: 3,390,429
Non-trainable params: 0
_________________________________________________________________


In [185]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '
            
            
    return output_text

In [186]:
def on_epoch_end(epoch, logs):
    seed_text = ""
    gen_words = 500

    print('Temp 0.2')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
    print('Temp 0.33')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
    print('Temp 0.5')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))
    print('Temp 1.0')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

    
    


In [188]:
train_model = True
if train_model:
    epochs = 1
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks = [callback], shuffle = True)

Epoch 1/1
93680/93680 [==============================] - 238s 3ms/step - loss: 6.5976
Temp 0.2


ValueError: cannot reshape array of size 0 into shape (1,20)

In [143]:
probs = model.predict(token_list, verbose=0)[0]

In [144]:
probs

array([2.1859702e-05, 2.9225013e-04, 5.3777482e-04, ..., 2.4494071e-05,
       2.2484226e-05, 2.3868801e-05], dtype=float32)

In [147]:
top_10_idx = np.flip(np.argsort(probs)[-10:])
top_10_probs = [probs[x] for x in top_10_idx]
top_10_words = tokenizer.sequences_to_texts([[x] for x in top_10_idx])

for prob, word in zip(top_10_probs, top_10_words):
    print('{:<6.1%} : {}'.format(prob, word))

1.8%   : duke
1.6%   : baron
1.3%   : man
0.7%   : fremen
0.7%   : old
0.7%   : way
0.6%   : voice
0.5%   : own
0.5%   : harkonnens
0.5%   : emperor


In [150]:
chosen_word = input()
            


seed_text += chosen_word + ' '
output_text += chosen_word + ' '

clear_output()

print (output_text)

duke


NameError: name 'output_text' is not defined

In [160]:
seed_text = "the frog and the snake walked into"
gen_words = 500
temp = 0.1

print (generate_text(seed_text, gen_words, model, seq_length, temp))

ValueError: cannot reshape array of size 7 into shape (1,20)